# Notebook E-tivity 3 CE4021 Task 2

Student name: Sean Meade

Student ID: 10128921

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [2]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [3]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

In [4]:
# Iterate over the labelled spam emails

def set_of_words_from_emails(emails):
    email_words = []

    for email in emails:
        words = email.split()
        for word in words:
            email_words.append(word)

    return set(email_words)


In [5]:
# for each word w in the entire training set, count how many of the spam emails contain w
def dict_count_words_in_emails(words, emails):

    # words is set of words
    # emails is a list of emails as strings

    words_count = {}
    for word in words:
        words_count[word] = 0
        for email in emails:
            if word in email:
                words_count[word] += 1
    
    return words_count

In [6]:
def prob_of_word_in_emails(word_count, emails):

    word_prob = {}
    email_count = len(emails)
    for word, count in word_count.items():
        word_prob[word] = (count + 1)/(email_count + 2)
    
    return word_prob

In [7]:
def prob_of_words_for(emails):
    words = set_of_words_from_emails(emails)
    word_count = dict_count_words_in_emails(words, emails)
    word_probs = prob_of_word_in_emails(word_count, emails)

    return word_probs

In [8]:
P_w_S = prob_of_words_for(previous_spam)

In [9]:
P_w_H = prob_of_words_for(previous_ham)

In [10]:
def probability_of_spam_and_ham(spam_emails, ham_emails):
    probs = {}

    total_email_count = len(spam_emails) + len(ham_emails)

    probs["prob_email_spam"] = len(spam_emails)/total_email_count
    probs["prob_email_ham"] = len(ham_emails)/total_email_count

    return probs

prob_of_spam_and_ham = probability_of_spam_and_ham(previous_spam, previous_ham)
prob_of_spam_and_ham
P_S = prob_of_spam_and_ham["prob_email_spam"]
P_H = prob_of_spam_and_ham["prob_email_ham"]

In [11]:
def list_of_words_from_emails(emails):
    email_words = []

    for email in emails:
        words = email.split()
        for word in words:
            email_words.append(word)

    return email_words

In [12]:
def remove_untrained_words(test_emails, type):
    trained_words = list_of_words_from_emails(previous_spam) + list_of_words_from_emails(previous_ham)
    set_of_email_words = set_of_words_from_emails([test_emails[type][0]])

    cross_over_words = []

    for word in set_of_email_words:
        if word in trained_words:
            cross_over_words.append(word)

    return cross_over_words

test_email_words = remove_untrained_words(new_emails, "spam")

In [13]:
h_words = set_of_words_from_emails(previous_ham)
h_word_count = dict_count_words_in_emails(h_words, previous_ham)
h_word_probs = prob_of_word_in_emails(h_word_count, previous_ham)
s_words = set_of_words_from_emails(previous_spam)
s_word_count = dict_count_words_in_emails(s_words, previous_spam)
s_word_probs = prob_of_word_in_emails(s_word_count, previous_spam)
P_w_H = prob_of_words_for(previous_ham)
P_w_S = prob_of_words_for(previous_spam)

P_S + P_H

1.0

<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Declaring the range you are choosing to experiment in the problem description like in Jason Colemans
Also from Jason that the expected value is actually different to the mean